# __Multithreading and its utilities__
In this tutorial, we will learn the basics of the Threading module and remember usages of threading and locks, mainly. The PyQt5 Threads will also be seen, since they are really useful and more convinient to use with a PyQt UI.

In [ ]:
%config IPCompleter.greedy=True

## _Threads_
In this example, I show two way of creating a thread. You can either create a function (threadLauncher1) and then create a Thread object with the function as a Target or create a new Object which inherits from treading.Thread which will already be a Thread in itself. Both methods can be used in different situations.

- Thread: Will enable the process of multiple function at the same time
- Daemon Threads: Will be killed at the end of the process even if the thread isn't done.

The ThreadLauncher1 is just a simple function, which we later bind to a Thread Object.
The ThreadLauncher2 is in itself a new Thread Object which has the function we need in it.
The ThreadLauncherFunction3 is an instance of our ThreadLauncher2 Object, but with Daemon activated.

Since the delays are not all equals, the following should happen:
1. Timer1, Timer2 and Timer 3 all start together.
2. Timer2 finishes first.
3. Timer1 finishes.
4. This allows the main to continue (because the .join() was waiting the timer1 to end)
5. The timer 3 shouldn't finish because the program has ended and it was a Daemon Thread.

In [2]:
import threading
import logging
import time

def threadLauncher1(name, delay):
    logging.info("Thread %s \t : Starting" %name)
    time.sleep(delay)
    logging.info("Thread %s \t : Finishing" %name)
    
    
class threadLauncher2(threading.Thread):
    def __init__(self, name, delay):
        super().__init__()
        self.name = name
        self.delay = delay
        
    def run(self):
        logging.info("Thread %s \t : Starting" % self.name)
        time.sleep(self.delay)
        logging.info("Thread %s \t : Finishing" % self.name)

        
class threadLauncher3(threading.Thread):
    def __init__(self, name, delay):
        super().__init__()
        self.name = name
        self.delay = delay
        
    def run(self):
        while True:
            logging.info('Thread %s running.' %self.name)
            time.sleep(self.delay)
    
if __name__ == "__main__":
    
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    
    threadLauncherFunction1 = threading.Thread(target=threadLauncher1, args=("Timer 1", 4,))
    threadLauncherFunction1.start()
    
    threadLauncherFunction2 = threadLauncher2("Timer 2", 2)
    threadLauncherFunction2.start()
    
    threadLauncherFunction3 = threadLauncher3("Timer 3", 1)
    threadLauncherFunction3.setDaemon(True)
    # threadLauncherFunction3.start()
    
    logging.info("Daemon setting is %s on %s" % (str(threadLauncherFunction3.isDaemon()), threadLauncherFunction3.name))
    
    threadLauncherFunction1.join()
    logging.info("Waited until Timer 1 had finished")
    logging.info("Timer3 should not be able to finish, since it is a daemon Thread")
    

15:30:15: Thread Timer 1 	 : Starting
15:30:15: Thread Timer 2 	 : Starting
15:30:15: Daemon setting is True on Timer 3
15:30:17: Thread Timer 2 	 : Finishing
15:30:19: Thread Timer 1 	 : Finishing
15:30:19: Waited until Timer 1 had finished
15:30:19: Timer3 should not be able to finish, since it is a daemon Thread


## _Locks_
Locks are objects in Python that facilitates the implementation of *Synchronisation* between threads. For example, two threads that make transactions in a bank account. They have to access the same variable, but you don,t wan't the threads to access it at the same time, as it would cause errors. But you still want the code to be separated in threads, as they might do other stuff while waiting, or constantly updating the amount they will transfer. Hence, a _Lock_ will make a thread wait while an other thread has released it. There are two types of locks: the *threading.Lock()* and *threading.semaphore()*. The semaphore is just like a normal lock, but with more control over how many threads can acces a specific variable an so on. Here we present an example of a situation where a Lock() is needed. Two operations (addition and multiplication) must always be made in sequential order because for some reason, the value of one is dependent of the output of the other one.

In [11]:

class Withdraw(threading.Thread):
    def __init__(self, account, amount):
        super().__init__()
        self.amount = amount
        self.account = account
        self.lock = threading.Lock()
        
        
    def run(self):
        while 1:
            self.lock.acquire()
            global account
            logging.info("Withdrawing %d from %s" % (self.amount, account))
            time.sleep(2)
            account -= self.amount
            self.lock.release()

class Deposit(threading.Thread):
    def __init__(self, account, amount):
        super().__init__()
        self.amount = amount
        self.account = account
        self.lock = threading.Lock()
        
        
    def run(self):
        while 1:
            self.lock.acquire()
            global account
            logging.info("Depositing %d from %s" % (self.amount, account))
            time.sleep(2)
            account += self.amount
            self.lock.release()

if __name__ == "__main__":
    
    account = 0
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    
    Withdrawer = Withdraw(account, 100)
    Depositer = Deposit(account, 100)
    Withdrawer.setDaemon(True)
    Depositer.setDaemon(True)
    
    Withdrawer.start()
    Depositer.start()
    
    for i in range(100):
        time.sleep(0.1)
        logging.info('Solde: %d' %account)
        

15:52:03: Withdrawing 100 from 0
15:52:03: Depositing 100 from 0
15:52:03: Solde: 0
15:52:03: Withdrawing 100 from -100
15:52:03: Depositing 100 from 0
15:52:03: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Withdrawing 100 from -100
15:52:04: Depositing 100 from 0
15:52:04: Solde: 0
15:52:04: Solde: 0
15:52:04: Depositing 100 from 61700
15:52:04: Withdrawing 100 from -61700
15:52:05: Solde: 0
15:52:05: Withdrawing 100 from -100
15:52:05: Depositing 100 from 0
15:52:05: Solde: 0
15:52:05: Solde: 0
15:52:05: Withdrawing 100 from -100
15:52:05: Depositing 100 from 0
15:52:05: Solde: 0
15:52:05: Withdrawing 100 from -100
15:52:05: Depositing 100 from 0
15:52:05: Solde: 0
15:52:05: Solde: 0
15:52:05: Solde: 0
15:52:05: Withdrawing 100 from -100
15:52:05: Depositing 100 from 0
15:52:05: Solde: 0
15:52:05: Withdrawing 100 from -100
15:52:05: Solde: -100
15:52:05: Depos

15:52:29: Depositing 100 from 0
15:52:29: Withdrawing 100 from -100
15:52:29: Depositing 100 from 0
15:52:30: Withdrawing 100 from -100
15:52:30: Depositing 100 from 0
15:52:31: Depositing 100 from 63000
15:52:31: Withdrawing 100 from -63000
15:52:31: Withdrawing 100 from -100
15:52:31: Depositing 100 from 0
15:52:31: Withdrawing 100 from -100
15:52:31: Depositing 100 from 0
15:52:31: Withdrawing 100 from -100
15:52:31: Depositing 100 from 0
15:52:31: Withdrawing 100 from -100
15:52:31: Depositing 100 from 0
15:52:31: Withdrawing 100 from -100
15:52:31: Depositing 100 from 0
15:52:32: Withdrawing 100 from -100
15:52:32: Depositing 100 from 0
15:52:33: Depositing 100 from 63100
15:52:33: Withdrawing 100 from -63100
15:52:33: Withdrawing 100 from -100
15:52:33: Depositing 100 from 0
15:52:33: Withdrawing 100 from -100
15:52:33: Depositing 100 from 0
15:52:33: Withdrawing 100 from -100
15:52:33: Depositing 100 from 0
15:52:33: Withdrawing 100 from -100
15:52:33: Depositing 100 from 0
15:5